<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    Binary Classification with Deep Learning: Titanic Survival Prediction
</div>

In [1]:
# ======================================================================= 
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# 
# 
# =======================================================================
# Module: Binary Classification with Neural Networks
# Topic: Implementation using Keras API
# =======================================================================
# Learning Objectives:
# 1. Design and implement neural network architecture for binary classification
# 2. Configure model compilation with appropriate loss functions and metrics
# 3. Train and evaluate classification models using the Titanic dataset
# 4. Understand the differences between regression and classification tasks
# =======================================================================
# Prerequisites:
# - Basic understanding of Python programming
# - Familiarity with neural network concepts
# - Knowledge of data preprocessing techniques
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2024
# =======================================================================

In [2]:
# 1. Environment Setup
# ------------------
import os  
from pathlib import Path
from pprint import pprint                     # This will be used for printing dicts in a nicer format
# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"   # if you have GPU

# Import necessary modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from layers import the Input and Dense
from tensorflow.keras.layers import Input, Dense

# Import the Sequential Model
from tensorflow.keras.models import Sequential

# import utils from the keras
from tensorflow.keras.utils import to_categorical

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Configuration
DATA_PATH = Path("../../datasets/classification/titanic.csv").resolve()

Author: Dr. Saad Laouadi

Last updated: 2024-12-10

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
matplotlib: 3.9.2
pandas    : 2.2.2
numpy     : 1.26.4
keras     : 3.6.0



## Project Overview

In this hands-on project, we will develop a deep learning classification model to predict passenger survival from the historic Titanic disaster. This project serves as an excellent introduction to binary classification problems, one of the fundamental tasks in machine learning and deep learning.

## Dataset Description
We will work with the famous Titanic dataset, which contains detailed information about passengers including:
- Demographic information (age, gender)
- Socio-economic features (passenger class, fare)
- Travel information (cabin, port of embarkation)
- Family relationships (siblings/spouses aboard, parents/children aboard)

Our target variable is binary:
    - Survived (1)
    - Did not survive (0)

## Technical Implementation
The project will utilize:
- Keras Sequential model development
- **Binary Cross-entropy** loss function
- **Stochastic Gradient Descent (SGD)** optimizer

## Model Architecture
We will construct a neural network with:
- An input layer matching our feature dimensions
- A hidden layer with `ReLU` activation
- An output layer with `sigmoid` activation for binary classification

## Learning Objectives

Through this project, you will learn:
1. How to prepare data for binary classification
2. Designing appropriate neural network architecture
3. Configuring loss functions and optimizers
4. Training and evaluating classification models

## Key Metrics
We will evaluate our model using:
- `Accuracy`: Percentage of correct predictions
- `Loss`: Categorical cross-entropy loss

In [3]:
# ==================================================== #
#        Load and Explore the data
# ==================================================== #
# Load the dataset
data = pd.read_csv(DATA_PATH)

# Display basic information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   survived                   891 non-null    int64  
 1   pclass                     891 non-null    int64  
 2   age                        891 non-null    float64
 3   sibsp                      891 non-null    int64  
 4   parch                      891 non-null    int64  
 5   fare                       891 non-null    float64
 6   male                       891 non-null    int64  
 7   age_was_missing            891 non-null    bool   
 8   embarked_from_cherbourg    891 non-null    int64  
 9   embarked_from_queenstown   891 non-null    int64  
 10  embarked_from_southampton  891 non-null    int64  
dtypes: bool(1), float64(2), int64(8)
memory usage: 70.6 KB


In [4]:
# ==================================================== #
#        Prepare the data for model
# ==================================================== #
# Separate features (predictors) from the target variable
predictors = data.drop(columns="survived").values
predictors = predictors.astype('float64')         # Convert to float64 for numerical stability

# Prepare the target variable
target = data['survived'].values

# Reshape the target to be 2d array
# target = target.reshape(-1, 1)

# Print shapes to verify the data structure
print(f"Features shape: {predictors.shape}")
print(f"Target shape: {target.shape}")
print(f"Sample of target data:\n{target[:5]}")

# Get the number of input features
n_cols = predictors.shape[1]
print(f"Number of input features: {n_cols}")

Features shape: (891, 10)
Target shape: (891,)
Sample of target data:
[0 1 1 1 0]
Number of input features: 10


In [5]:
# ==================================================== #
#        Build Model Architecture
# ==================================================== #
# Initialize the sequential model
model = Sequential()

# Add the input layer with shape matching our features
model.add(Input(shape=(n_cols,)))

# Add hidden layer with 32 neurons and ReLU activation
model.add(Dense(32, activation='relu'))

# Add output layer with 2 neurons (binary classification) and softmax activation
model.add(Dense(1, activation='sigmoid'))

# Display model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 385 (1.50 KB)

 Trainable params: 385 (1.50 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# ==================================================== #
#        Train the model
# ==================================================== #

# Configure the model with optimizer, loss function, and metrics
model.compile(
    optimizer='sgd',                       # Stochastic Gradient Descent optimizer
    loss='binary_crossentropy',            # Standard loss for classification
    metrics=['accuracy']                   # Track accuracy during training
)

# Train the model
model.fit(
    predictors,                            # Input features
    target,                                # Target variable
    epochs=25,                             # Number of training cycles
    batch_size=16,                         # Number of samples per gradient update
    validation_split=0.2                   # Use 20% of data for validation
)

Epoch 1/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5512 - loss: 3.0066 - val_accuracy: 0.7821 - val_loss: 0.5993
Epoch 2/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6512 - loss: 0.7488 - val_accuracy: 0.7430 - val_loss: 0.5674
Epoch 3/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5920 - loss: 0.7983 - val_accuracy: 0.6425 - val_loss: 0.7129
Epoch 4/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6618 - loss: 0.6354 - val_accuracy: 0.6872 - val_loss: 0.5536
Epoch 5/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6646 - loss: 0.6084 - val_accuracy: 0.7430 - val_loss: 0.5055
Epoch 6/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6840 - loss: 0.6154 - val_accuracy: 0.7542 - val_loss: 0.5298
Epoch 7/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6897 - loss: 0.6110 - val_accuracy: 0.7374 - val_loss: 0.5023
Epoch 8/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6938 - loss: 0.6087 - val_accuracy: 0.6425 - val_loss: